# Kapitel 7 - Parametrisierung

## 7.1. Kapitelübersicht <a class="anchor" id="7-1"/>

TODO

<b>Abschnittsübersicht</b><br>

[7.1. Kapitelübersicht](#7-1)<br>

## 7.2. Parameter in Scikit learn

TODO<br>
Bis jetzt hatten wir nur die standard parameter von scikit learn bei unseren klassifizierungsverfahren genutzt. selbst hatten wir keine parameter übergeben. durch gezieltes parameter tuning können klassifizierungsverfahren jedoch noch einmal verbessert werden. in diesem abschnitt werden wir uns zunächst erneut die verfahren naive bayes und log reg angucken und deren parameter in der dokumentation. intuitionen hatten wir schon in den einzelnen Kapiteln besprochen.

### 7.2.1. Parameter von Multinomial Naive Bayes

Implementieren wir zunächst ein weiteres Mal das <b>Multinomial Naive Bayes</b> Klassifizierungsverfahren.

In [4]:
import pandas as pd
corpus = pd.read_csv("tutorialdata/corpora/wikicorpus_v2.csv", index_col=0)

In [7]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
import numpy as np

labels = LabelEncoder().fit_transform(corpus["category"])
vector  = TfidfVectorizer().fit_transform(corpus["text"])


X_train, X_test, y_train, y_test = train_test_split(vector, 
                                                    labels, 
                                                    test_size=0.2, 
                                                    train_size=0.8,
                                                    random_state=42)

# Multinomial Naive Bayes
mnb_classifier = MultinomialNB()
mnb = mnb_classifier.fit(X_train, y_train)

# cross validation des Trainingsdatensatzes
mnb_scores = cross_val_score(mnb_classifier, vector, labels, cv=3)
mnb_mean = np.mean(mnb_scores)

print("Der Mittelwert der cross validation bei der  Klassifizierung " 
      + f" mit Multinomial Naive Bayes ist {str(np.around(mnb_mean, decimals=3))}."
      + "\n")


# F1-score des Testdatensatzes
y_pred = mnb_classifier.predict(X_test)
mnb_f1 = f1_score(y_test, y_pred, average="micro")

print("Der F1-score für die Klassifizierung mit Multinomial Naive Bayes ist "
      + f"{str(np.around(mnb_f1, decimals=3))}.")

Der Mittelwert der cross validation bei der  Klassifizierung  mit Multinomial Naive Bayes ist 0.859.

Der F1-score für die Klassifizierung mit Multinomial Naive Bayes ist 0.87.


Schauen wir uns zunächst die möglichen Parameter von Multinomial Naive Bayes in der <a href="https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html">Dokumentation</a> an. Es gibt drei Parameter:
- `alpha` (default = 1.0)
- `fit_prior` (default = True)
- `class_prior` (default = None)<br>

Wir werden uns nur `alpha` anschauen. Ist `alpha = 1`, wir <b>Laplace Smoothing</b> angewandt (siehe Kapitel 3), d.h. jede Worthäufigkeit wird um 1 erhöht. Ist `alpha < 1`, wird <b>Lidstone Smoothing</b> angewandt, das im Grunde das Gleiche ist. Ist `alpha = 0`, wird gar kein Smoothing angewandt.

TODO hier anwenden

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

https://machinelearningmastery.com/difference-between-a-parameter-and-a-hyperparameter/

Schauen wir uns zunächst die möglichen Parameter von Multinomial Naive Bayes in der <a href="https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html">Dokumentation</a> an. 

In [ ]:
from sklearn.model_selection import GridSearchCV

alphas = np.array([0.000001, 0.00001, 0.0001, 0.001, 0.01, 
                0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
parameters = {'alpha': alphas, 
              'fit_prior' : [True, False], 
              'class_prior ' : [None, [.1,.9],[.2, .8]]}


grid = GridSearchCV(estimator=mnb_classifier, param_grid=dict(alpha=alphas), cv=5)
grid.fit(X_train, y_train)
print(grid)
# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_estimator_.alpha)